In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [3]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [4]:
X = np.append(s25,h25,axis=0)

In [5]:
y = np.append(s25_labels,h25_labels,axis=0)

In [6]:
X.shape

(1142, 6250, 19)

In [15]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [16]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [17]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [18]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 44ms/step - loss: 0.7573 - accuracy: 0.4808
Epoch 2/50
8/8 [==============================] - 0s 47ms/step - loss: 0.7138 - accuracy: 0.5126
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7107 - accuracy: 0.5170
Epoch 4/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7180 - accuracy: 0.5115
Epoch 5/50
8/8 [==============================] - 0s 47ms/step - loss: 0.7214 - accuracy: 0.5071
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7087 - accuracy: 0.5126
Epoch 7/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7062 - accuracy: 0.5027
Epoch 8/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7042 - accuracy: 0.5235
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7099 - accuracy: 0.5060
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7109 - accuracy: 0.5016
Epoch 11/50
8/8 [============

8/8 [==============================] - 0s 50ms/step - loss: 0.7030 - accuracy: 0.5279
Epoch 18/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7043 - accuracy: 0.4962
Epoch 19/50
8/8 [==============================] - 0s 47ms/step - loss: 0.7016 - accuracy: 0.5235
Epoch 20/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7016 - accuracy: 0.5159
Epoch 21/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6965 - accuracy: 0.5235
Epoch 22/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6964 - accuracy: 0.5345
Epoch 23/50
8/8 [==============================] - 0s 47ms/step - loss: 0.7008 - accuracy: 0.5192
Epoch 24/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7076 - accuracy: 0.5093
Epoch 25/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7016 - accuracy: 0.5104
Epoch 26/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7001 - accuracy: 0.4951
Epoch 27/50
8/8 [===============

Epoch 43/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7001 - accuracy: 0.4885
Epoch 44/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6922 - accuracy: 0.5257
Epoch 45/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6987 - accuracy: 0.5257
Epoch 46/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6970 - accuracy: 0.5323
Epoch 47/50
8/8 [==============================] - 0s 47ms/step - loss: 0.6910 - accuracy: 0.5170
Epoch 48/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6962 - accuracy: 0.5422
Epoch 49/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7017 - accuracy: 0.4973
Epoch 50/50
8/8 [==============================] - 0s 47ms/step - loss: 0.6931 - accuracy: 0.5203
 >#5: 61.572 
[61.57205104827881, 98.25327396392822, 61.57205104827881, 61.57205104827881, 61.57205104827881]
 Accuracy: 68.908% (+/-14.672) 
